In [11]:
import os
import open3d as o3d
import numpy as np
import plotly
import json
import sklearn

# Compute auc rog

In [12]:
our_results = {}
for file_name in os.listdir('jsons'):
    file_idx = int(file_name.split('.')[0])
    
    with open(f'jsons/{file_name}', 'r') as inf:
        data = json.load(inf)
    
    dop = data['door_open_percent']
    our_results[file_idx] = dop
    
our_results

{49: 66,
 70: 75,
 50: 66,
 72: 0,
 68: 8,
 19: 8,
 64: 0,
 23: 0,
 30: 8,
 67: 0,
 84: 41,
 57: 0,
 6: 8,
 24: 16,
 90: 33,
 17: 0,
 41: 16,
 14: 0,
 60: 16,
 40: 25,
 85: 25,
 29: 8,
 99: 41,
 83: 33,
 31: 8,
 9: 16,
 71: 16,
 34: 25,
 27: 16,
 63: 0,
 62: 25,
 45: 25,
 73: 0,
 8: 16,
 4: 8,
 95: 41,
 69: 0,
 61: 33,
 48: 0,
 52: 8,
 88: 50,
 2: -1,
 25: 16,
 15: 0,
 20: 8,
 94: 41,
 36: 0,
 75: 8,
 51: 8,
 91: 58,
 7: 16,
 54: 16,
 37: 75,
 42: 16,
 92: 58,
 79: 8,
 98: 25,
 87: 33,
 26: 16,
 22: 8,
 13: 0,
 81: 58,
 82: 25,
 80: 25,
 53: 16,
 16: 16,
 100: 50,
 38: 50,
 39: 25,
 46: 16,
 44: 33,
 96: 33,
 11: 16,
 76: 33,
 55: 16,
 65: 0,
 59: 25,
 97: 50,
 1: -1,
 32: 16,
 35: 16,
 56: 0,
 93: 66,
 74: 0,
 18: 58,
 33: 0,
 78: 41,
 12: 16,
 43: 25,
 58: 8,
 66: 0,
 10: 0,
 28: 16,
 89: 58,
 47: 8,
 21: 25,
 3: -1,
 77: 8,
 5: 8,
 86: 58}

In [17]:
trues_str = '''0001 UNKNOWN
0002 UNKNOWN
0003 UNKNOWN
0004 SEMI
0005 CLOSED
0006 CLOSED
0007 CLOSED
0008 CLOSED
0009 SEMI
0010 CLOSED
0011 CLOSED
0012 SEMI
0013 SEMI
0014 SEMI
0015 CLOSED
0016 CLOSED
0017 CLOSED
0018 OPEN
0019 CLOSED
0020 CLOSED
0021 SEMI
0022 CLOSED
0023 CLOSED
0024 SEMI
0025 SEMI
0026 SEMI
0027 SEMI
0028 SEMI
0029 SEMI
0030 SEMI
0031 SEMI
0032 SEMI
0033 SEMI
0034 SEMI
0035 SEMI
0036 SEMI
0037 SEMI
0038 SEMI
0039 SEMI
0040 SEMI
0041 SEMI
0042 SEMI
0043 SEMI
0044 SEMI
0045 CLOSED
0046 SEMI
0047 CLOSED
0048 CLOSED
0049 OPEN
0050 OPEN
0051 SEMI
0052 SEMI
0053 SEMI
0054 SEMI
0055 SEMI
0056 SEMI
0057 CLOSED
0058 CLOSED
0059 SEMI
0060 SEMI
0061 SEMI
0062 CLOSED
0063 SEMI
0064 CLOSED
0065 CLOSED
0066 CLOSED
0067 CLOSED
0068 SEMI
0069 CLOSED
0070 OPEN
0071 SEMI
0072 SEMI
0073 CLOSED
0074 CLOSED
0075 CLOSED
0076 CLOSED
0077 SEMI
0078 SEMI
0079 CLOSED
0080 SEMI
0081 OPEN
0082 OPEN
0083 OPEN
0084 OPEN
0085 OPEN
0086 OPEN
0087 SEMI
0088 OPEN
0089 SEMI
0090 OPEN
0091 OPEN
0092 OPEN
0093 OPEN
0094 SEMI
0095 OPEN
0096 OPEN
0097 OPEN
0098 OPEN
0099 OPEN
0100 OPEN'''

trues_dict = {}
for line in trues_str.split('\n'):
    idx, status = line.split(' ')
    idx = int(idx)
    is_closed = status == 'CLOSED'
    trues_dict[idx] = is_closed
print(idx, trues_dict)

100 {1: False, 2: False, 3: False, 4: False, 5: True, 6: True, 7: True, 8: True, 9: False, 10: True, 11: True, 12: False, 13: False, 14: False, 15: True, 16: True, 17: True, 18: False, 19: True, 20: True, 21: False, 22: True, 23: True, 24: False, 25: False, 26: False, 27: False, 28: False, 29: False, 30: False, 31: False, 32: False, 33: False, 34: False, 35: False, 36: False, 37: False, 38: False, 39: False, 40: False, 41: False, 42: False, 43: False, 44: False, 45: True, 46: False, 47: True, 48: True, 49: False, 50: False, 51: False, 52: False, 53: False, 54: False, 55: False, 56: False, 57: True, 58: True, 59: False, 60: False, 61: False, 62: True, 63: False, 64: True, 65: True, 66: True, 67: True, 68: False, 69: True, 70: False, 71: False, 72: False, 73: True, 74: True, 75: True, 76: True, 77: False, 78: False, 79: True, 80: False, 81: False, 82: False, 83: False, 84: False, 85: False, 86: False, 87: False, 88: False, 89: False, 90: False, 91: False, 92: False, 93: False, 94: False,

In [18]:
keys_list = []
trues_list = []
preds_list = []
for k,v in trues_dict.items():
    keys_list.append(k)
    trues_list.append(v)
    r = our_results[k]
    if r == -1:
        continue
    preds_list.append((100-r)/100)
    
print(keys_list)
print(trues_list)
print(preds_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[False, False, False, False, True, True, True, True, False, True, True, False, False, False, True, True, True, False, True, True, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, True, False, False, False, False, False, False, False, False, True, True, False, False, False, True, False, True, True, True, True, False, True, False, False, False, True, True, True, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False

In [19]:
sklearn.metrics.roc_auc_score(trues_list, preds_list)

ValueError: Found input variables with inconsistent numbers of samples: [100, 97]